# Topic Modeling
In this exercise, we will do topic modeling with gensim. Use the [topics and transformations tutorial](https://radimrehurek.com/gensim/auto_examples/core/run_topics_and_transformations.html) as a reference.

In [1]:
import os
from collections import defaultdict

import gensim
import nltk

For tokenizing words and stopword removal, download the NLTK punkt tokenizer and stopwords list.

In [2]:
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\iamarfur\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iamarfur\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

First, we load the [Lee Background Corpus](https://hekyll.services.adelaide.edu.au/dspace/bitstream/2440/28910/1/hdl_28910.pdf) included with gensim that contains 300 news articles of the Australian Broadcasting Corporation.

In [3]:
from gensim.test.utils import datapath
train_file = datapath('lee_background.cor')
articles_orig = open(train_file).read().splitlines()

Preprocess the text by lowercasing, removing stopwords, stemming, and removing rare words.

In [4]:
# define stopword list
stopwords = set(nltk.corpus.stopwords.words('english'))
stopwords = stopwords | {'\"', '\'', '\'\'', '`', '``', '\'s'}

# initialize stemmer
stemmer = nltk.stem.PorterStemmer()

def preprocess(article):
    # tokenize
    article = nltk.word_tokenize(article)

    # lowercase all words
    article = [word.lower() for word in article]

    # remove stopwords
    article = [word for word in article if word not in stopwords]

    # optional: stem
    # article = [stemmer.stem(word) for word in article]
    return article

articles = [preprocess(article) for article in articles_orig]

# create the dictionary and corpus objects that gensim uses for topic modeling
dictionary = gensim.corpora.Dictionary(articles)

# remove words that occur in less than 2 documents, or more than 50% of documents
dictionary.filter_extremes(no_below=2, no_above=0.5)
temp = dictionary[0]  # load the dictionary by calling it once
corpus_bow = [dictionary.doc2bow(article) for article in articles]


Now we create a TF-IDF model and transform the corpus into TF-IDF vectors.

In [5]:
model_tfidf = gensim.models.TfidfModel(corpus_bow)
corpus_tfidf = model_tfidf[corpus_bow]
print(corpus_tfidf[0])

[(0, 0.045163832296308125), (1, 0.049004990699027966), (2, 0.09398031720792203), (3, 0.06797874731615453), (4, 0.08637534553463992), (5, 0.10158528888120417), (6, 0.058872481173046734), (7, 0.045871696227162966), (8, 0.04660732651093343), (9, 0.03476708703034139), (10, 0.09174339245432593), (11, 0.06379342938648586), (12, 0.08097953226203827), (13, 0.08637534553463992), (14, 0.06576958891547403), (15, 0.05748249959948285), (16, 0.07679421433236962), (17, 0.09398031720792203), (18, 0.04197717742438698), (19, 0.06379342938648586), (20, 0.09398031720792203), (21, 0.07679421433236962), (22, 0.08097953226203827), (23, 0.058872481173046734), (24, 0.05497796237027076), (25, 0.05497796237027076), (26, 0.07337456058875615), (27, 0.05497796237027076), (28, 0.08637534553463992), (29, 0.058872481173046734), (30, 0.062005775644911734), (31, 0.08637534553463992), (32, 0.09398031720792203), (33, 0.04737299069698862), (34, 0.07048328454536662), (35, 0.09398031720792203), (36, 0.09398031720792203), (37

Now we train an [LDA model](https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html) with 10 topics on the TF-IDF corpus. Save it to a variable `model_lda`.

In [6]:
model_lda = gensim.models.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=10)

Let's inspect the first 5 topics of our model.

In [7]:
model_lda.print_topics(5)

[(6,
  '0.002*"palestinian" + 0.002*"arafat" + 0.002*"israeli" + 0.001*"hospital" + 0.001*"army" + 0.001*"hih" + 0.001*"use" + 0.001*"assa" + 0.001*"abloy" + 0.001*"new"'),
 (9,
  '0.003*"palestinian" + 0.002*"israeli" + 0.002*"arafat" + 0.002*"zinni" + 0.002*"israel" + 0.002*"fire" + 0.001*"south" + 0.001*"weather" + 0.001*"mr" + 0.001*"test"'),
 (5,
  '0.003*"palestinian" + 0.002*"israeli" + 0.002*"gaza" + 0.002*"sharon" + 0.002*"radio" + 0.002*"strip" + 0.001*"hill" + 0.001*"markets" + 0.001*"arafat" + 0.001*"zimbabwe"'),
 (7,
  '0.002*"qantas" + 0.002*"palestinian" + 0.002*"mr" + 0.002*"us" + 0.001*"taliban" + 0.001*"suicide" + 0.001*"workers" + 0.001*"afghanistan" + 0.001*"australian" + 0.001*"bin"'),
 (2,
  '0.002*"taliban" + 0.002*"afghanistan" + 0.002*"government" + 0.002*"williams" + 0.002*"australian" + 0.002*"bin" + 0.002*"fire" + 0.002*"laden" + 0.002*"australia" + 0.002*"detainees"')]

We see the 5 topics with the highest importance. For each topic, the 10 most important words are shown, together with their coefficient of "alignment" to the topic.

## Document Similarity
We now use our LDA model to compare the similarity of new documents (*queries*) to documents in our collection.

First, create an index of the news articles in our corpus. Use the `MatrixSimilarity` transformation as described in gensim's [similarity queries tutorial](https://radimrehurek.com/gensim/auto_examples/core/run_similarity_queries.html).

In [8]:
from gensim import similarities
index = similarities.MatrixSimilarity(model_lda[corpus_tfidf])

Now, write a function that takes a query string as input and returns the LDA representation for it. Make sure to apply the same preprocessing as we did to the documents.

In [9]:
def get_lda_vector(text):
    text = preprocess(text)
    bow = dictionary.doc2bow(text)
    tfidf = model_tfidf[bow]
    lda = model_lda[tfidf]
    return lda

lda_vector = get_lda_vector('A new bill sparked massive protests in Israel, as it would massively limit the powers of the judiciary.')

Print the top 5 most similar documents, together with their similarities, using your index created above.

In [10]:
document_similarities = index[lda_vector]
for doc_idx, doc_similarity in sorted(enumerate(document_similarities), key=lambda x: x[1], reverse=True)[:5]:
    print(doc_similarity, articles_orig[doc_idx])

0.99852127 New statistics released by the Cancer Council reveal some alarming trends about lung cancer. The report reveals lung cancer now rivals breast cancer as the leading cause of cancer death among women. The statistics, covering the past three decades, also show high numbers of lung cancer cases and deaths in poorer areas such as some parts of Sydney and far western New South Wales. The Cancer Council's chief executive officer, Andrew Penman, says another trend is in the type of lung cancer which is believed to be caused by changes in the design of cigarettes. "Modern cigarettes are particularly dangerous to the peripheral airways and they're causing lung cancers deeper in the lungs than traditional cigarettes say 40 years ago," he said. About 2,200 people die from lung cancer in New South Wales each year. 
0.9979083 A pay freeze dispute involving Qantas and its maintenance workers will remain unresolved over the Christmas period. The parties failed to reach agreement during talk

Run your code again, now training an LDA model with 100 topics. Do you see a qualitative difference in the top-5 most similar documents?